# Init

In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
print("CURRENT FOLDER:")
%pwd

CURRENT FOLDER:


'c:\\max\\Google Drive Max\\Max Shared\\Master Data Science\\2020-05 Capstone A\\msds_capstone\\analysis\\fullRuns'

In [79]:
import os
# import glob
import sys
sys.path.append(os.environ['DEV_AUTOTS'])
sys.path.append(os.environ['CAPSTONE_PYTHON_SOURCE'])
# folder = os.environ['CAPSTONE_DATA']

In [80]:
## Sourcing the .py with all the code##
t='"'+os.environ['CAPSTONE_PYTHON_SOURCE']+'\\fullRun\\fullRun.py'+'"'
%run -i $t

In [ ]:
from ETL.ETL import loadDataset

# Parameters

In [81]:
#Parameters

ProdCats = ['SUP PREM WHISKEY']
TOP_PRODUCTS = 3  # How many products to consider in the category
TOP_SIMILAR = 1   # Get TOP_SIMILAR most similar products
RUN_PERIODS = [1,2,3] # this will run AutoTS and Optimzer for speicfic periods (1 = last, 2 = second to last)

CV_NUM = 10         # number of Cross Valudations  10
OPT_CALLS = 100     # number of calls for the optimizer process   100
OPT_RND_STARTS = 20 # number of random calls for the optimizer process  20

LOG_TRANSFORM = True # Take log of 9L cases to smooth out peaks and valleys
ZERO_ADDER = 0.1 

RESAMPLE_FREQ = 'M'


# Pricing changes every 4 weeks
if RESAMPLE_FREQ == 'M':    FORECAST_PERIOD = 1 #last 4 months as test 
if RESAMPLE_FREQ == 'W':    FORECAST_PERIOD = 4 
if RESAMPLE_FREQ == '2W':   FORECAST_PERIOD = 2 

# Seasonal Period
if RESAMPLE_FREQ == 'M':    SEASONAL_PERIOD = 12  # Yearly
if RESAMPLE_FREQ == 'W':    SEASONAL_PERIOD = 13 # Quarterly (we can also take yearly = 52, but SARIMAX becomes too slow)
if RESAMPLE_FREQ == '2W':   SEASONAL_PERIOD = 13 # This becomes problematic --> for quarterly, should we take 6 biweekly periods or 7 bi-weekly periods. Instead I just took half yearly period  

COL_TIME = 'WeekDate'
COL_PREDS = ['9L Cases'] #Demand
COL_PRICE= ['Dollar Sales per 9L Case'] #Price


# Full Run

In [82]:
dataRaw = loadDataset(version=4)
ProdCat = 'SUP PREM WHISKEY' #'ECONOMY VODKA'
ChainMasters =  [''] #+  dataRaw['Chain Master'].unique().tolist() 
display(ChainMasters,ProdCats)

['']

['SUP PREM WHISKEY']

In [83]:
(full_models,full_opt,full_summary,full_chain_summary)  = fullRun(ProdCat = ProdCat
                                                           ,ChainMasters = ChainMasters
                                                           ,dataRaw = dataRaw
                                                           ,savePKL=True
                                                           ,saveXLS=True)

Parameters being used...
TOP_PRODUCTS = 2
TOP_SIMILAR = 1
RUN_PERIODS = [1]
ProdCat: SUP PREM WHISKEY
ChainMasters: ['']


========================== (2020-08-31 17:21:01.898866)
===== Products List  =====
==== SUP PREM WHISKEY ====
============  ============



========================== (2020-08-31 17:21:07.212674)
======== AUTO-TS =========
==== SUP PREM WHISKEY ====
============  ============
=========== 1 ============



================== (2020-08-31 17:21:07.213674)
==== GET DATA ====
========  ========

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '1']
% of weeks without a purchase: 0.0
resampling to  M



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1.75L
--------------------------------------------------
Exogenous 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.




========================= (2020-08-31 17:24:13.352035)
==== Multivar P0+Sim ====
===========  ============



[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.8min finished




=================== (2020-08-31 17:27:04.425927)
==== COMPLETED ====
========  =========



========================== (2020-08-31 17:27:04.430926)
======= OPTIMIZER ========
==== SUP PREM WHISKEY ====
============  ============
=========== 1 ============



=========================== (2020-08-31 17:27:04.430926)
==== Running Optimizer ====
==== SUP PREM WHISKEY =====
============  =============
============ 1 ============



================== (2020-08-31 17:27:04.433927)
==== GET DATA ====
========  ========

resampling to  M
Decoder: {'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L'}



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1L
--------------------------------------------------
Exogenous Price Columns: ['0', '1']
% of weeks without a purchase: 0.0
resampling to  M



--------------------------------------------------
Product: JACK DANIELS BLK WHSKY  1.75L
--------------------------------------------------
Exogeno

# Print out

## Models

In [84]:
full_models[outputColsModels]

,Chain Master,Product,Period to Last,White Noise,Naive Best Type,Naive Best RMSE,P0 Best Model Name,P0 Best Model RMSE,P0+Sim Best Model Name,P0+Sim Best Model RMSE
1,,JACK DANIELS BLK WHSKY 1.75L,1,False,last,0.324455,ML,0.224917,SARIMAX,0.142141
0,,JACK DANIELS BLK WHSKY 1L,1,True,mean,0.043169,SARIMAX,0.011244,SARIMAX,0.007738


## Optimizer

### Summary by Chain

In [85]:
full_chain_summary

,Chain Master,P0 Non-Opt Revenue,P0 Opt Revenue,P0 Opt Diff,P0 Opt Diff Ratio,P0+Sim Non-Opt Revenue,P0+Sim Opt Revenue,P0+Sim Opt Diff,P0+Sim Opt Diff Ratio
0,,94651.0,89770.0,-4881.0,0.948432,150999.0,153075.0,2076.0,1.013748


In [58]:

print("WRITE this value in the summary table in Git hub --->",round(full_chain_summary[full_chain_summary['Chain Master']=='']['P0+Sim Opt Diff Ratio'].item(),4))

WRITE this value in the summary table in Git hub ---> 1.0137


Link to the Git Hub Summary Table: https://github.com/ngupta23/msds_capstone/issues/84

### Summary by Chain and Product

In [87]:
full_summary

,Chain Master,Product,P0 Non-Opt Revenue,P0 Opt Revenue,P0 Opt Diff,P0 Opt Diff Ratio,P0+Sim Non-Opt Revenue,P0+Sim Opt Revenue,P0+Sim Opt Diff,P0+Sim Opt Diff Ratio
0,,JACK DANIELS BLK WHSKY 1.75L,23364.0,21792.0,-1572.0,0.932717,80391.0,82908.0,2517.0,1.031309
1,,JACK DANIELS BLK WHSKY 1L,71287.0,67978.0,-3309.0,0.953582,70608.0,70167.0,-441.0,0.993754


### P0 Optimizer

In [88]:
full_opt[outputColsOptP0]

,Chain Master,Product,Period to Last,P0 Non-Opt Price,P0 Non-Opt Demand,P0 Non-Opt Revenue,P0 Non-Opt Chain Master Revenue,P0 Optimal Price,P0 Demand,P0 Revenue,P0 Chain Master Revenue
1,,JACK DANIELS BLK WHSKY 1.75L,1,185.65,126,23364,94651,173.16,126,21792,89770
0,,JACK DANIELS BLK WHSKY 1L,1,229.81,310,71287,94651,228.22,298,67978,89770


### P0+Sim Optimizer

In [89]:
full_opt[outputColsOptP0Sim]


,Chain Master,Product,Period to Last,P0+Sim Non-Opt Price,P0+Sim Non-Opt Demand,P0+Sim Non-Opt Revenue,P0+Sim Non-Opt Chain Master Revenue,P0+Sim Optimal Price,P0+Sim Demand,P0+Sim Revenue,P0+Sim Chain Master Revenue
1,,JACK DANIELS BLK WHSKY 1.75L,1,185.65,433,80391,150999,173.16,479,82908,153075
0,,JACK DANIELS BLK WHSKY 1L,1,229.81,307,70608,150999,228.22,307,70167,153075


### Naive

In [90]:
full_opt[outputColsOptNaive]

,Chain Master,Product,Period to Last,Actual Price,Actual Demand,Actual Revenue,Actual Chain Master Revenue,Naive Prices,Naive Demand,Naive Revenue,Naive Chain Master Revenue
1,,JACK DANIELS BLK WHSKY 1.75L,1,185.650112,389.61,72331.14,207733.62,185.650112,40.950000,7602.372072,74847.602904
0,,JACK DANIELS BLK WHSKY 1L,1,229.811232,589.19,135402.48,207733.62,229.811232,292.610723,67245.230832,74847.602904
